# Import libraries

In [1]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd
import ast
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Import data

In [2]:
x_y_train = pd.read_csv("x_y_train.csv", low_memory=False, index_col="row_index")

x_test = pd.read_csv("x_test.csv", low_memory=False, index_col="row_index")

# Preprocess train

## Piezo qualification:

We only keep the rows where the measure is qualified as "Correct" or not yet qualified:

In [3]:
x_y_train["piezo_qualification"].value_counts()

piezo_qualification
Correcte        2718869
Non qualifié      93199
Incorrecte         9462
Incertaine         8786
Name: count, dtype: int64

In [4]:
x_y_train = x_y_train[x_y_train["piezo_qualification"].isin(["Correcte", "Non qualifié"])].drop(columns=["piezo_qualification"])

## Irrelevant columns:

In [5]:
columns_to_drop = [
    "piezo_station_department_code",
    "piezo_station_update_date",
    "piezo_station_department_name",
    "piezo_station_commune_code_insee",
    "piezo_station_pe_label",
    "piezo_station_bss_code",
    "piezo_station_commune_name",
    "piezo_station_bss_id",
    "piezo_bss_code",
    "piezo_obtention_mode",
    "piezo_continuity_code",
    "piezo_continuity_name",
    "piezo_producer_code",
    "piezo_producer_name",
    "piezo_measure_nature_code",
    "meteo_id",
    "meteo_name",
    "meteo_date",
    "meteo_DRR",
    "meteo_temperature_avg_threshold",
    "meteo_temperature_min_50cm",
    "meteo_pressure_avg",
    "meteo_pression_maxi",
    "meteo_wind_speed_avg_2m",
    "meteo_wind_max_2m",
    "meteo_wind_direction_max_inst_2m",
    "meteo_time_wind_max_2m",
    "meteo_wetting_duration",
    "meteo_sunshine_duration",
    "meteo_longitude",
    "meteo_latitude",
    "meteo_radiation",
    "meteo_radiation_direct",
    "meteo_sunshine_%",
    "meteo_radiation_IR",
    "meteo_radiation_UV_max",
    "meteo_cloudiness",
    "meteo_cloudiness_height",
    "meteo_if_snow",
    "meteo_if_fog",
    "meteo_if_thunderstorm",
    "meteo_if_sleet",
    "meteo_if_hail",
    "meteo_if_dew",
    "meteo_if_black_ice",
    "meteo_if_snow_ground",
    "meteo_if_frost",
    "meteo_if_smoke",
    "meteo_if_mist",
    "meteo_if_lightning",
    "meteo_evapotranspiration_Monteith",
    "meteo_radiation_UV",
    "meteo_snow_height",
    "meteo_snow_thickness_max",
    "meteo_snow_thickness_6h",
    "hydro_station_code",
    "hydro_observation_date_elab",
    "hydro_status_label",
    "hydro_method_label",
    "hydro_qualification_label",
    "hydro_longitude",
    "hydro_latitude",
    "hydro_hydro_quantity_elab",
    "prelev_structure_code_0",
    "prelev_volume_obtention_mode_label_0",
    "prelev_commune_code_insee_0",
    "prelev_structure_code_1",
    "prelev_volume_obtention_mode_label_1",
    "prelev_commune_code_insee_1",
    "prelev_structure_code_2",
    "prelev_volume_obtention_mode_label_2",
    "prelev_commune_code_insee_2"
]

In [6]:
x_y_train = x_y_train.drop(columns=columns_to_drop)

## Date:

For the date, we apply cyclical encoding:

In [7]:
x_y_train["piezo_measurement_date"]

row_index
0          2020-01-01
1          2020-01-01
2          2020-01-01
3          2020-01-01
4          2020-01-01
              ...    
3294080    2023-05-31
3294081    2023-05-31
3294082    2023-05-31
3294083    2023-05-31
3294084    2023-05-31
Name: piezo_measurement_date, Length: 2812068, dtype: object

In [8]:
x_y_train["piezo_measurement_date"] = pd.to_datetime(x_y_train["piezo_measurement_date"])
x_y_train["day_of_year"] = x_y_train["piezo_measurement_date"].dt.dayofyear
x_y_train["day_sin"] = np.sin(2 * np.pi * x_y_train["day_of_year"] / 365)
x_y_train["day_cos"] = np.cos(2 * np.pi * x_y_train["day_of_year"] / 365)
x_y_train["month"] = x_y_train["piezo_measurement_date"].dt.month
x_y_train["year"] = x_y_train["piezo_measurement_date"].dt.year

In [9]:
x_y_train = x_y_train[x_y_train["piezo_measurement_date"].dt.month.between(3, 10)]

In [10]:
x_y_train = x_y_train.drop(columns=["piezo_measurement_date", "day_of_year"])

## BDLISA code:

In [11]:
eh_code_prefix_to_category = {
    "101": "Sands", "104": "Sands", "107": "Limestones", "110": "Marls", 
    "113": "Limestones", "117": "Other", "119": "Other", "121": "Other", 
    "123": "Marls", "125": "Marls", "127": "Sands", "131": "Limestones", 
    "133": "Marls", "135": "Limestones", "137": "Marls", "139": "Limestones", 
    "141": "Marls", "143": "Gravel", "144": "Sands", "145": "Other", 
    "147": "Gravel", "149": "Limestones", "151": "Massif", "153": "Massif", 
    "154": "Massif", "155": "Massif", "156": "Massif", "158": "Massif", 
    "159": "Massif", "160": "Massif", "161": "Sands", "163": "Massif", 
    "164": "Other", "165": "Other", "167": "Other", "169": "Massif", 
    "170": "Massif", "171": "Massif", "173": "Other", "174": "Basement", 
    "175": "Basement", "177": "Basement", "179": "Basement", "181": "Basement", 
    "183": "Basement", "185": "Basement", "186": "Basement", "187": "Basement", 
    "189": "Other", "191": "Other", "193": "Other", "195": "Other", 
    "197": "Other", "199": "Other", "201": "Basement", "203": "Basement", 
    "205": "Basement", "206": "Basement", "207": "Basement", "208": "Basement", 
    "211": "Other", "221": "Alluvions", "222": "Alluvions", "223": "Other", 
    "225": "Paleogeological Epochs", "226": "Gravel", "227": "Marls", 
    "230": "Gravel", "231": "Basement", "233": "Volcanic", "306": "Gravel", 
    "308": "Sands", "312": "Sands", "316": "Sands", "318": "Paleogeological Epochs", 
    "320": "Limestones", "322": "Paleogeological Epochs", "324": "Limestones", 
    "326": "Marls", "328": "Marls", "330": "Marls", "332": "Massif", 
    "334": "Gravel", "338": "Other", "340": "Marls", "342": "Other", 
    "344": "Limestones", "346": "Marls", "348": "Limestones", "350": "Gravel", 
    "352": "Limestones", "356": "Marls", "358": "Limestones", "359": "Limestones", 
    "360": "Marls", "362": "Limestones", "364": "Geological Epochs", "366": "Gravel", 
    "368": "Gravel", "370": "Basement", "372": "Basement", "374": "Basement", 
    "400": "Limestones", "402": "Limestones", "404": "Massif", "502": "Other", 
    "505": "Other", "507": "Other", "509": "Paleogeological Epochs", "513": "Limestones", 
    "515": "Limestones", "516": "Other", "517": "Limestones", "519": "Marls", 
    "521": "Other", "523": "Limestones", "525": "Other", "527": "Basement", 
    "529": "Paleogeological Epochs", "531": "Limestones", "533": "Limestones", 
    "534": "Limestones", "548": "Marls", "561": "Other", "563": "Other", 
    "565": "Limestones", "567": "Massif", "569": "Limestones", "571": "Limestones", 
    "573": "Geological Epochs", "577": "Marls", "581": "Massif", "583": "Marls", 
    "585": "Marls", "600": "Paleogeological Epochs", "602": "Basement", 
    "604": "Basement", "621": "Limestones", "631": "Limestones", "643": "Marls", 
    "647": "Alluvions", "651": "Limestones", "657": "Other", "671": "Alluvions", 
    "679": "Marls", "681": "Marls", "691": "Basement", "693": "Basement", 
    "699": "Basement", "710": "Alluvions", "712": "Alluvions", "714": "Alluvions", 
    "716": "Alluvions", "718": "Alluvions", "719": "Alluvions", "720": "Alluvions", 
    "901": "Basement", "902": "Sands", "910": "Alluvions", "912": "Alluvions", 
    "914": "Alluvions", "916": "Alluvions", "918": "Alluvions", "920": "Alluvions", 
    "922": "Alluvions", "924": "Alluvions", "926": "Alluvions", "928": "Alluvions", 
    "930": "Alluvions", "932": "Alluvions", "935": "Alluvions", "936": "Alluvions", 
    "937": "Alluvions", "938": "Alluvions", "940": "Alluvions", "942": "Alluvions", 
    "944": "Alluvions", "946": "Alluvions", "948": "Alluvions", "949": "Alluvions", 
    "950": "Alluvions", "952": "Alluvions", "954": "Alluvions", "971": "Volcanic", 
    "972": "Volcanic", "974": "Volcanic", "976": "Volcanic"
}

In [12]:
def map_code_to_category(code):
    prefix = str(code)[:3]
    return eh_code_prefix_to_category.get(prefix, "Other")

In [13]:
def map_codes_in_list(codes_list):
    return [map_code_to_category(code) for code in codes_list]

In [14]:
def safe_eval(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return []
    else:
        return []

In [15]:
x_y_train["mapped_category"] = x_y_train["piezo_station_bdlisa_codes"].apply(lambda x: map_codes_in_list(safe_eval(x)))

In [16]:
mlb = MultiLabelBinarizer()
x_y_train[mlb.classes_] = mlb.fit_transform(x_y_train["mapped_category"])

In [17]:
x_y_train = x_y_train.drop(columns=["piezo_station_bdlisa_codes", "mapped_category"])

## Piezo status and measure:

In [18]:
piezo_status_to_id = {
    "Donnée brute": 0,
    "Donnée contrôlée niveau 1": 1,
    "Donnée contrôlée niveau 2": 2,
    "Donnée interprétée": 3
}

In [19]:
piezo_measure_nature_name_to_id = {
    "Inconnue": np.nan,
    "Naturel": 1,
    "Influencé": 2,
    "Dynamique": 3,
    "Sec": 4
}

In [20]:
x_y_train["piezo_status"] = x_y_train["piezo_status"].map(piezo_status_to_id)
x_y_train["piezo_measure_nature_name"] = x_y_train["piezo_measure_nature_name"].map(piezo_measure_nature_name_to_id)

## Distance computation:

In [21]:
prelev_columns = [col for col in x_y_train.columns if col.startswith("prelev_")]
prelev = x_y_train[prelev_columns]

In [22]:
numeric_columns = prelev.dtypes[((prelev.dtypes=="float64")) | ((prelev.dtypes=="int64"))].index.values.tolist()

In [23]:
prelev_copy = prelev[numeric_columns]
prelev_copy["prelev_usage_label_0"] = prelev["prelev_usage_label_0"].astype("category")
prelev_copy["prelev_usage_label_1"] = prelev["prelev_usage_label_1"].astype("category")
prelev_copy["prelev_usage_label_2"] = prelev["prelev_usage_label_2"].astype("category")
prelev_copy["prelev_other_volume_sum"] = prelev["prelev_other_volume_sum"]

prelev_copy["distance_0"] = (x_y_train["piezo_station_longitude"] - x_y_train["prelev_longitude_0"])**2 + (x_y_train["piezo_station_latitude"] - x_y_train["prelev_latitude_0"])**2
prelev_copy["distance_1"] = (x_y_train["piezo_station_longitude"] - x_y_train["prelev_longitude_1"])**2 + (x_y_train["piezo_station_latitude"] - x_y_train["prelev_latitude_1"])**2
prelev_copy["distance_2"] = (x_y_train["piezo_station_longitude"] - x_y_train["prelev_longitude_2"])**2 + (x_y_train["piezo_station_latitude"] - x_y_train["prelev_latitude_2"])**2
prelev_copy = prelev_copy.drop(columns=["prelev_longitude_0", "prelev_longitude_1", "prelev_longitude_2", "prelev_latitude_0", "prelev_latitude_1", "prelev_latitude_2"])

prelev_copy["prelev_all_volume_sum"] = x_y_train["prelev_volume_0"] + x_y_train["prelev_volume_1"] + x_y_train["prelev_volume_2"] + x_y_train["prelev_other_volume_sum"]

C:\Users\alexa\AppData\Local\Temp\ipykernel_17484\1652206598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prelev_copy["prelev_usage_label_0"] = prelev["prelev_usage_label_0"].astype("category")
C:\Users\alexa\AppData\Local\Temp\ipykernel_17484\1652206598.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prelev_copy["prelev_usage_label_1"] = prelev["prelev_usage_label_1"].astype("category")


In [24]:
x_y_train = x_y_train.drop(columns=prelev_columns)

prelev_columns = [col for col in prelev_copy.columns]

x_y_train[prelev_columns] = prelev_copy[prelev_columns]

## Insee columns:

In [25]:
insee_columns = [
    "insee_%_agri",
    "insee_med_living_level",
    "insee_%_ind",
    "insee_%_const"
]

In [26]:
x_y_train[insee_columns] = x_y_train[insee_columns].apply(pd.to_numeric, errors="coerce")

## Deviation data:

In [27]:
deviation_data = {
    2020: {"temperature": 1.15, "precipitation": 0.97},
    2021: {"temperature": -0.01, "precipitation": 0.99},
    2022: {"temperature": 1.6, "precipitation": 0.76},
    2023: {"temperature": 1.4, "precipitation": 1.035},
}

In [28]:
x_y_train["mean_annual_temperature_deviation"] = x_y_train["year"].map(
    lambda year: deviation_data[year]["temperature"] if year in deviation_data else None
)
x_y_train["mean_annual_precipitation_deviation"] = x_y_train["year"].map(
    lambda year: deviation_data[year]["precipitation"] if year in deviation_data else None
)

# Preprocess test

## Piezo qualification:

In [29]:
x_test["piezo_qualification"].value_counts()

piezo_qualification
Correcte        578616
Non qualifié     26930
Incertaine        3025
Incorrecte        2637
Name: count, dtype: int64

In [30]:
x_test = x_test.drop(columns=["piezo_qualification"])

## Irrelevant columns:

In [31]:
columns_to_drop = [
    "piezo_station_department_code",
    "piezo_station_update_date",
    "piezo_station_department_name",
    "piezo_station_commune_code_insee",
    "piezo_station_pe_label",
    "piezo_station_bss_code",
    "piezo_station_commune_name",
    "piezo_station_bss_id",
    "piezo_bss_code",
    "piezo_obtention_mode",
    "piezo_continuity_code",
    "piezo_continuity_name",
    "piezo_producer_code",
    "piezo_producer_name",
    "piezo_measure_nature_code",
    "meteo_id",
    "meteo_name",
    "meteo_date",
    "meteo_DRR",
    "meteo_temperature_avg_threshold",
    "meteo_temperature_min_50cm",
    "meteo_pressure_avg",
    "meteo_pression_maxi",
    "meteo_wind_speed_avg_2m",
    "meteo_wind_max_2m",
    "meteo_wind_direction_max_inst_2m",
    "meteo_time_wind_max_2m",
    "meteo_wetting_duration",
    "meteo_sunshine_duration",
    "meteo_longitude",
    "meteo_latitude",
    "meteo_radiation",
    "meteo_radiation_direct",
    "meteo_sunshine_%",
    "meteo_radiation_IR",
    "meteo_radiation_UV_max",
    "meteo_cloudiness",
    "meteo_cloudiness_height",
    "meteo_if_snow",
    "meteo_if_fog",
    "meteo_if_thunderstorm",
    "meteo_if_sleet",
    "meteo_if_hail",
    "meteo_if_dew",
    "meteo_if_black_ice",
    "meteo_if_snow_ground",
    "meteo_if_frost",
    "meteo_if_smoke",
    "meteo_if_mist",
    "meteo_if_lightning",
    "meteo_evapotranspiration_Monteith",
    "meteo_radiation_UV",
    "meteo_snow_height",
    "meteo_snow_thickness_max",
    "meteo_snow_thickness_6h",
    "hydro_station_code",
    "hydro_observation_date_elab",
    "hydro_status_label",
    "hydro_method_label",
    "hydro_qualification_label",
    "hydro_longitude",
    "hydro_latitude",
    "hydro_hydro_quantity_elab",
    "prelev_structure_code_0",
    "prelev_volume_obtention_mode_label_0",
    "prelev_commune_code_insee_0",
    "prelev_structure_code_1",
    "prelev_volume_obtention_mode_label_1",
    "prelev_commune_code_insee_1",
    "prelev_structure_code_2",
    "prelev_volume_obtention_mode_label_2",
    "prelev_commune_code_insee_2"
]

In [32]:
x_test = x_test.drop(columns=columns_to_drop)

## Date:

For the date, we apply cyclical encoding:

In [33]:
x_test["piezo_measurement_date"]

row_index
2331795    2022-06-01
2331796    2022-06-01
2331797    2022-06-01
2331798    2022-06-01
2331799    2022-06-01
              ...    
3610818    2023-09-30
3610819    2023-09-30
3610820    2023-09-30
3610821    2023-09-30
3610822    2023-09-30
Name: piezo_measurement_date, Length: 611208, dtype: object

In [34]:
x_test["piezo_measurement_date"] = pd.to_datetime(x_test["piezo_measurement_date"])
x_test["day_of_year"] = x_test["piezo_measurement_date"].dt.dayofyear
x_test["day_sin"] = np.sin(2 * np.pi * x_test["day_of_year"] / 365)
x_test["day_cos"] = np.cos(2 * np.pi * x_test["day_of_year"] / 365)
x_test["month"] = x_test["piezo_measurement_date"].dt.month
x_test["year"] = x_test["piezo_measurement_date"].dt.year

In [35]:
x_test = x_test[x_test["piezo_measurement_date"].dt.month.between(3, 10)]

In [36]:
x_test = x_test.drop(columns=["piezo_measurement_date", "day_of_year"])

## BDLISA code:

In [37]:
eh_code_prefix_to_category = {
    "101": "Sands", "104": "Sands", "107": "Limestones", "110": "Marls", 
    "113": "Limestones", "117": "Other", "119": "Other", "121": "Other", 
    "123": "Marls", "125": "Marls", "127": "Sands", "131": "Limestones", 
    "133": "Marls", "135": "Limestones", "137": "Marls", "139": "Limestones", 
    "141": "Marls", "143": "Gravel", "144": "Sands", "145": "Other", 
    "147": "Gravel", "149": "Limestones", "151": "Massif", "153": "Massif", 
    "154": "Massif", "155": "Massif", "156": "Massif", "158": "Massif", 
    "159": "Massif", "160": "Massif", "161": "Sands", "163": "Massif", 
    "164": "Other", "165": "Other", "167": "Other", "169": "Massif", 
    "170": "Massif", "171": "Massif", "173": "Other", "174": "Basement", 
    "175": "Basement", "177": "Basement", "179": "Basement", "181": "Basement", 
    "183": "Basement", "185": "Basement", "186": "Basement", "187": "Basement", 
    "189": "Other", "191": "Other", "193": "Other", "195": "Other", 
    "197": "Other", "199": "Other", "201": "Basement", "203": "Basement", 
    "205": "Basement", "206": "Basement", "207": "Basement", "208": "Basement", 
    "211": "Other", "221": "Alluvions", "222": "Alluvions", "223": "Other", 
    "225": "Paleogeological Epochs", "226": "Gravel", "227": "Marls", 
    "230": "Gravel", "231": "Basement", "233": "Volcanic", "306": "Gravel", 
    "308": "Sands", "312": "Sands", "316": "Sands", "318": "Paleogeological Epochs", 
    "320": "Limestones", "322": "Paleogeological Epochs", "324": "Limestones", 
    "326": "Marls", "328": "Marls", "330": "Marls", "332": "Massif", 
    "334": "Gravel", "338": "Other", "340": "Marls", "342": "Other", 
    "344": "Limestones", "346": "Marls", "348": "Limestones", "350": "Gravel", 
    "352": "Limestones", "356": "Marls", "358": "Limestones", "359": "Limestones", 
    "360": "Marls", "362": "Limestones", "364": "Geological Epochs", "366": "Gravel", 
    "368": "Gravel", "370": "Basement", "372": "Basement", "374": "Basement", 
    "400": "Limestones", "402": "Limestones", "404": "Massif", "502": "Other", 
    "505": "Other", "507": "Other", "509": "Paleogeological Epochs", "513": "Limestones", 
    "515": "Limestones", "516": "Other", "517": "Limestones", "519": "Marls", 
    "521": "Other", "523": "Limestones", "525": "Other", "527": "Basement", 
    "529": "Paleogeological Epochs", "531": "Limestones", "533": "Limestones", 
    "534": "Limestones", "548": "Marls", "561": "Other", "563": "Other", 
    "565": "Limestones", "567": "Massif", "569": "Limestones", "571": "Limestones", 
    "573": "Geological Epochs", "577": "Marls", "581": "Massif", "583": "Marls", 
    "585": "Marls", "600": "Paleogeological Epochs", "602": "Basement", 
    "604": "Basement", "621": "Limestones", "631": "Limestones", "643": "Marls", 
    "647": "Alluvions", "651": "Limestones", "657": "Other", "671": "Alluvions", 
    "679": "Marls", "681": "Marls", "691": "Basement", "693": "Basement", 
    "699": "Basement", "710": "Alluvions", "712": "Alluvions", "714": "Alluvions", 
    "716": "Alluvions", "718": "Alluvions", "719": "Alluvions", "720": "Alluvions", 
    "901": "Basement", "902": "Sands", "910": "Alluvions", "912": "Alluvions", 
    "914": "Alluvions", "916": "Alluvions", "918": "Alluvions", "920": "Alluvions", 
    "922": "Alluvions", "924": "Alluvions", "926": "Alluvions", "928": "Alluvions", 
    "930": "Alluvions", "932": "Alluvions", "935": "Alluvions", "936": "Alluvions", 
    "937": "Alluvions", "938": "Alluvions", "940": "Alluvions", "942": "Alluvions", 
    "944": "Alluvions", "946": "Alluvions", "948": "Alluvions", "949": "Alluvions", 
    "950": "Alluvions", "952": "Alluvions", "954": "Alluvions", "971": "Volcanic", 
    "972": "Volcanic", "974": "Volcanic", "976": "Volcanic"
}

In [38]:
def map_code_to_category(code):
    prefix = str(code)[:3]
    return eh_code_prefix_to_category.get(prefix, "Other")

In [39]:
def map_codes_in_list(codes_list):
    return [map_code_to_category(code) for code in codes_list]

In [40]:
def safe_eval(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return []
    else:
        return []

In [41]:
x_test["mapped_category"] = x_test["piezo_station_bdlisa_codes"].apply(lambda x: map_codes_in_list(safe_eval(x)))

In [42]:
mlb = MultiLabelBinarizer()
x_test[mlb.classes_] = mlb.fit_transform(x_test["mapped_category"])

In [43]:
x_test = x_test.drop(columns=["piezo_station_bdlisa_codes", "mapped_category"])

## Piezo status and measure:

In [44]:
piezo_status_to_id = {
    "Donnée brute": 0,
    "Donnée contrôlée niveau 1": 1,
    "Donnée contrôlée niveau 2": 2,
    "Donnée interprétée": 3
}

In [45]:
piezo_measure_nature_name_to_id = {
    "Inconnue": np.nan,
    "Naturel": 1,
    "Influencé": 2,
    "Dynamique": 3,
    "Sec": 4
}

In [46]:
x_test["piezo_status"] = x_test["piezo_status"].map(piezo_status_to_id)
x_test["piezo_measure_nature_name"] = x_test["piezo_measure_nature_name"].map(piezo_measure_nature_name_to_id)

## Distance computation:

In [47]:
prelev_columns = [col for col in x_test.columns if col.startswith("prelev_")]
prelev = x_test[prelev_columns]

In [48]:
numeric_columns = prelev.dtypes[((prelev.dtypes=="float64")) | ((prelev.dtypes=="int64"))].index.values.tolist()

In [49]:
prelev_copy = prelev[numeric_columns]
prelev_copy["prelev_usage_label_0"] = prelev["prelev_usage_label_0"].astype("category")
prelev_copy["prelev_usage_label_1"] = prelev["prelev_usage_label_1"].astype("category")
prelev_copy["prelev_usage_label_2"] = prelev["prelev_usage_label_2"].astype("category")
prelev_copy["prelev_other_volume_sum"] = prelev["prelev_other_volume_sum"]

prelev_copy["distance_0"] = (x_test["piezo_station_longitude"] - x_test["prelev_longitude_0"])**2 + (x_test["piezo_station_latitude"] - x_test["prelev_latitude_0"])**2
prelev_copy["distance_1"] = (x_test["piezo_station_longitude"] - x_test["prelev_longitude_1"])**2 + (x_test["piezo_station_latitude"] - x_test["prelev_latitude_1"])**2
prelev_copy["distance_2"] = (x_test["piezo_station_longitude"] - x_test["prelev_longitude_2"])**2 + (x_test["piezo_station_latitude"] - x_test["prelev_latitude_2"])**2
prelev_copy = prelev_copy.drop(columns=["prelev_longitude_0", "prelev_longitude_1", "prelev_longitude_2", "prelev_latitude_0", "prelev_latitude_1", "prelev_latitude_2"])

prelev_copy["prelev_all_volume_sum"] = x_test["prelev_volume_0"] + x_test["prelev_volume_1"] + x_test["prelev_volume_2"] + x_test["prelev_other_volume_sum"]

C:\Users\alexa\AppData\Local\Temp\ipykernel_17484\2781368820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prelev_copy["prelev_usage_label_0"] = prelev["prelev_usage_label_0"].astype("category")
C:\Users\alexa\AppData\Local\Temp\ipykernel_17484\2781368820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prelev_copy["prelev_usage_label_1"] = prelev["prelev_usage_label_1"].astype("category")


In [50]:
x_test = x_test.drop(columns=prelev_columns)

prelev_columns = [col for col in prelev_copy.columns]

x_test[prelev_columns] = prelev_copy[prelev_columns]

## Insee columns:

In [51]:
insee_columns = [
    "insee_%_agri",
    "insee_med_living_level",
    "insee_%_ind",
    "insee_%_const"
]

In [52]:
x_test[insee_columns] = x_test[insee_columns].apply(pd.to_numeric, errors="coerce")

## Deviation data:

In [53]:
deviation_data = {
    2020: {"temperature": 1.15, "precipitation": 0.97},
    2021: {"temperature": -0.01, "precipitation": 0.99},
    2022: {"temperature": 1.6, "precipitation": 0.76},
    2023: {"temperature": 1.4, "precipitation": 1.035},
}

In [54]:
x_test["mean_annual_temperature_deviation"] = x_test["year"].map(
    lambda year: deviation_data[year]["temperature"] if year in deviation_data else None
)
x_test["mean_annual_precipitation_deviation"] = x_test["year"].map(
    lambda year: deviation_data[year]["precipitation"] if year in deviation_data else None
)

# Split data

In [55]:
x_y_train.shape

(1761874, 74)

In [56]:
x_test.shape

(611208, 73)

In [57]:
y_train = x_y_train["piezo_groundwater_level_category"]

x_train = x_y_train.drop(columns=["piezo_groundwater_level_category"])

In [58]:
groundwater_cat_to_id = {
    "Very High": 4,
    "High": 3,
    "Average": 2,
    "Low": 1,
    "Very Low": 0
}

y_train = y_train.map(groundwater_cat_to_id)

In [59]:
x_train, x_true, y_train, y_true = train_test_split(x_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# Model

In [60]:
model = xgb.XGBRegressor(
    objective="reg:squarederror",
    learning_rate=0.1,
    max_depth=10,
    n_estimators=700,
    min_child_weight=1,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=2.0,
    reg_lambda=1.0,
    random_state=42,
    enable_categorical=True
)

model.fit(x_train, y_train)

y_pred = np.clip(np.round(model.predict(x_true)).astype(int), 0, 4)

print(f"Accuracy: {accuracy_score(y_pred=y_pred, y_true=y_true):.4f}")
print(f"F1-Score (Micro): {f1_score(y_pred=y_pred, y_true=y_true, average="micro"):.4f}")
print(f"F1-Score (Macro): {f1_score(y_pred=y_pred, y_true=y_true, average="macro"):.4f}")
print(f"F1-Score (Weighted): {f1_score(y_pred=y_pred, y_true=y_true, average="weighted"):.4f}")

Accuracy: 0.8862
F1-Score (Micro): 0.8862
F1-Score (Macro): 0.8892
F1-Score (Weighted): 0.8869


In [61]:
y_pred = np.clip(np.round(model.predict(x_test)).astype(int), 0, 4)

In [62]:
y_pred = pd.DataFrame({
    "row_index": x_test.index,
    "piezo_groundwater_level_category": y_pred
})

groundwater_id_to_cat = {
    4: "Very High",
    3: "High",
    2: "Average",
    1: "Low",
    0: "Very Low"
}

y_pred["piezo_groundwater_level_category"] = y_pred["piezo_groundwater_level_category"].map(groundwater_id_to_cat)

In [63]:
y_test = pd.read_csv("y_test.csv", low_memory=False)

merged = pd.merge(y_pred, y_test, on="row_index", suffixes=("_pred", "_test"))

accuracy = accuracy_score(merged["piezo_groundwater_level_category_pred"], merged["piezo_groundwater_level_category_test"]) * 100

print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 60.55%
